In [1]:
import json
import os
import argparse
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from models import EncoderRNN, DecoderRNN, S2VTAttModel, S2VTModel
from dataloader import VideoDataset
import misc.utils as utils
from misc.cocoeval import suppress_stdout_stderr, COCOScorer

from pandas.io.json import json_normalize

In [2]:
args = {'recover_opt': 'data/save/opt_info.json', 'saved_model': 'data/save/model_50.pth', 'dump_json': 1, 'results_path': 'results/', 'dump_path': 0, 'gpu': '0', 'batch_size': 25, 'sample_max': 1, 'temperature': 1.0, 'beam_size': 1}


In [3]:
opt = json.load(open(args["recover_opt"]))
for k, v in args.items():
        opt[k] = v
os.environ['CUDA_VISIBLE_DEVICES'] = opt["gpu"]

In [6]:
dataset = VideoDataset(opt, "test")

vocab size is  16860
number of train videos:  6501
number of val videos:  500
number of test videos:  2999
load feats from ['data/feats/resnet152']
max sequence length in data is 28


In [9]:
opt["vocab_size"] = dataset.get_vocab_size()

In [10]:
opt["seq_length"] = dataset.max_len

In [12]:
opt["model"]

'S2VTAttModel'

In [17]:
encoder = EncoderRNN(opt["dim_vid"], opt["dim_hidden"], bidirectional=opt["bidirectional"],
                             input_dropout_p=opt["input_dropout_p"], rnn_dropout_p=opt["rnn_dropout_p"]);
decoder = DecoderRNN(opt["vocab_size"], opt["max_len"], opt["dim_hidden"], opt["dim_word"],
                             input_dropout_p=opt["input_dropout_p"],
                             rnn_dropout_p=opt["rnn_dropout_p"], bidirectional=opt["bidirectional"]);
model = S2VTAttModel(encoder, decoder).cuda()

/home/chongke/anaconda3/lib/python3.6/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [18]:
model

S2VTAttModel(
  (encoder): EncoderRNN(
    (vid2hid): Linear(in_features=2048, out_features=512, bias=True)
    (input_dropout): Dropout(p=0.2, inplace=False)
    (rnn): GRU(512, 512, batch_first=True, dropout=0.5, bidirectional=0)
  )
  (decoder): DecoderRNN(
    (input_dropout): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(16860, 512)
    (attention): Attention(
      (linear1): Linear(in_features=1024, out_features=512, bias=True)
      (linear2): Linear(in_features=512, out_features=1, bias=False)
    )
    (rnn): GRU(1024, 512, batch_first=True, dropout=0.5)
    (out): Linear(in_features=512, out_features=16860, bias=True)
  )
)

In [19]:
model.load_state_dict(torch.load(opt["saved_model"]))

<All keys matched successfully>

In [20]:
crit = utils.LanguageModelCriterion()

/home/chongke/anaconda3/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [21]:
model.encoder.rnn.bidirectional = bool(model.encoder.rnn.bidirectional)

In [23]:
model.encoder.rnn.bidirectional

False